# Note
As other nltk versions exist some bugs in calculating bleu, the the experiment should be ran in nltk 3.6.1 

In [ ]:
!pip install nltk==3.6.1
!pip install scipy

# The result of different combinations on $TLC_{Dedup}$

## utils function

In [1]:
import sys
sys.path.append("../..")
sys.path.append("../../metrics")
from metrics.evaluate import read_to_list
from metrics.evaluate import read_to_list
from bleu.codenn_bleu import codenn_smooth_bleu
from bleu.codenn_bleu import codenn_smooth_bleu
from bleu.google_bleu import compute_bleu
from bleu.rencos_bleu import Bleu as recos_bleu
import prettytable as pt
import os
import numpy as np


In [2]:
def get_mean_std(arr):
    arr_mean = np.mean(arr)
    arr_std = np.std(arr, ddof=1)
    return arr_mean ,  arr_std
def show_dict(all_bleu):
        tb = pt.PrettyTable()
        tb.field_names = all_bleu.keys()
        tb.add_row(all_bleu.values())
        print(tb)
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction        
def get_codenn_score(refs, preds):
    r_str_list = []
    p_str_list = []
    for r, p in zip(refs, preds):
        if len(r[0]) == 0 or len(p) == 0:
            continue
        r_str_list.append([" ".join([str(token_id) for token_id in r[0]])])
        p_str_list.append(" ".join([str(token_id) for token_id in p]))
    try:
        bleu_list = codenn_smooth_bleu(r_str_list, p_str_list)
    except:
        bleu_list = [0, 0, 0, 0]
    codenn_bleu = bleu_list[0]
    codenn_bleu = round(codenn_bleu,4)
    return codenn_bleu
def get_bleu_dm(refs,preds):
    sentence_bleu0 = [sentence_bleu(ref, pred) for ref, pred in zip(refs, preds)]
    sentence_bleu0 = np.mean(sentence_bleu0)
    sentence_bleu0 = round(sentence_bleu0 * 100, 4)
    return sentence_bleu0 
def get_bleu_fc(refs,preds):
    c_bleu4 = corpus_bleu(refs, preds, weights=(0.25, 0.25, 0.25, 0.25))
    c_bleu4 = round(c_bleu4 * 100, 4)
    return   c_bleu4
def get_bleu_dc(refs,preds):
    all_score = 0.0
    count = 0
    for r, p in zip(refs, preds):
        # nltk bug: https://github.com/nltk/nltk/issues/2204
        if len(p) == 1:
            continue
        score = nltk.translate.bleu(r, p, smoothing_function=SmoothingFunction().method4)
        all_score += score
        count += 1
    emse_bleu = round(all_score / count * 100, 4)
    return emse_bleu
def get_bleu_cn(refs,preds):
    r_str_list = []
    p_str_list = []
    for r, p in zip(refs, preds):
        if len(r[0]) == 0 or len(p) == 0:
            continue
        r_str_list.append([" ".join([str(token_id) for token_id in r[0]])])
        p_str_list.append(" ".join([str(token_id) for token_id in p]))
    try:
        bleu_list = codenn_smooth_bleu(r_str_list, p_str_list)
    except:
        bleu_list = [0, 0, 0, 0]
    codenn_bleu = bleu_list[0]
    codenn_bleu = round(codenn_bleu,4)
    return codenn_bleu
def get_bleu_ncs(refs,preds):
    google_bleu4 = [compute_bleu([ref], [pred], smooth=True)[0] for ref, pred in zip(refs, preds)]
    google_bleu4 = np.mean(google_bleu4)
    google_bleu4 = round(google_bleu4 * 100, 4)
    return google_bleu4
def get_bleu_rc(refs,preds):
    res = {k: [" ".join(v)] for k, v in enumerate(preds)}
    gts = {k: [" ".join(v[0])] for k, v in enumerate(refs)}
    _, scores_Bleu = recos_bleu(4).compute_score(gts, res)
    rencos_bleu4 =round(np.mean(scores_Bleu[3])*100, 4)
    return rencos_bleu4

In [3]:
def get_diff_combination_scores(approaches,data_dir,refs_filename,get_bleu_score):
    for approach in approaches:
        combination_result_dict = {}
        latex_diff_combination_result = {}
        for combination in all_data_processing_combinations:
#             refs_filename = os.path.join(data_dir, approach, combination,"test.gold")
            preds_filename = os.path.join(data_dir, approach,combination, "test.pred")
            preds = read_to_list(preds_filename)
            refs = read_to_list(refs_filename)
            refs = [[t] for t in refs[:len( preds)]]
            codenn_bleu = get_bleu_score(refs, preds)
            combination_result_dict[combination] = codenn_bleu
#             latex_diff_combination_result[combination] = "&%.2f "%codenn_bleu
        all_bleu = {key:round(value, 2) for key, value in combination_result_dict.items()}
        tb = pt.PrettyTable()
        tb.field_names = all_bleu.keys()
        tb.add_row(all_bleu.values())
        print(30*"*")
        print(approach)
#         print(" ".join(list(latex_diff_combination_result.values())))
        print(tb)

In [4]:
data_dir = "data_processing_result"
refs_filename = os.path.join(data_dir,"test.gold")
approaches = ["codenn", "astattgru", "rencos", "ncs" ]

In [5]:
all_data_processing_combinations = ['P0000', 'P0001', 'P0010', 'P0011', 'P0100', 'P0101', 'P0110', 'P0111', 'P1000', 'P1001', 'P1010', 'P1011', 'P1100', 'P1101', 'P1110', 'P1111']


In [6]:
get_diff_combination_scores(approaches ,data_dir,refs_filename,get_bleu_dc)

******************************
codenn
+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| P0000 | P0001 | P0010 | P0011 | P0100 | P0101 | P0110 | P0111 | P1000 | P1001 | P1010 | P1011 | P1100 | P1101 | P1110 | P1111 |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|  7.06 |  7.1  |  6.98 |  7.25 |  7.54 |  7.01 |  7.43 |  7.06 |  7.22 |  7.19 |  7.24 |  7.4  |  7.06 |  7.34 |  7.02 |  7.05 |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
******************************
astattgru
+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| P0000 | P0001 | P0010 | P0011 | P0100 | P0101 | P0110 | P0111 | P1000 | P1001 | P1010 | P1011 | P1100 | P1101 | P1110 | P1111 |
+-------+--

## significant test 

In [7]:
import numpy as np
import scipy.stats as stats
def get_four_operations(data):
    r0 = data[8:]
    r1 = data[:8]
    s0 = data[:4] + data[8:12]
    s1 = data[4:8] + data[12:]
    f0 = [data[i] for i in [2, 3, 6, 7, 10, 11, 14, 15]]
    f1 = [data[i] for i in [0, 1, 4, 5, 8, 9, 12, 13]]
    l0 = [data[2 * i] for i in range(8)]
    l1 = [data[2 * i + 1] for i in range(8)]
    return r0, r1, s0, s1, f0, f1, l0, l1


def get_t_test_and_MWW_test(approach_data):
    r0, r1, s0, s1, f0, f1, l0, l1 = [], [], [], [], [], [], [], []
    x1_new, r1_new, s0_new, s1_new, f0_new, f1_new, l0_new, l1_new = get_four_operations(approach_data)
    r0.extend(x1_new) 
    r1.extend(r1_new)
    s0.extend(s0_new)
    s1.extend(s1_new)
    f0.extend(f0_new)
    f1.extend(f1_new)
    l0.extend(l0_new)
    l1.extend(l1_new)
    p_val_t_dict = {} 
    p_mwu_val_dict = {} 
    avg = {"R0":np.mean(r0), "R1":np.mean(r1),"S0":np.mean(s0),"S1":np.mean(s1),
           "F0":np.mean(f0),"F1":np.mean(f1),"L0":np.mean(l0),"L1":np.mean(l1),}
    avg = {key:str(round(item,2))  for key,item in avg.items()}
    tb = pt.PrettyTable()
    tb.field_names = avg.keys()
    tb.add_row(avg.values())
    print(tb)
#     print("& ".join(list(avg.values())))
    # R 
    _, p_val_t_test = stats.ttest_ind(r0, r1, equal_var=False)
    _, p_val_wwu_test = stats.mannwhitneyu(r0, r1, alternative='two-sided') 
    p_val_t_dict["R"] = "%.4f" % p_val_t_test 
    p_mwu_val_dict["R"] = "%.4f" % p_val_wwu_test
    # S
    _, p_val_t_test = stats.ttest_ind(s0, s1, equal_var=False)
    _, p_val_wwu_test = stats.mannwhitneyu(s0, s1, alternative='two-sided')
    p_val_t_dict["S"]= "%.4f" % p_val_t_test
    p_mwu_val_dict["S"] ="%.4f" % p_val_wwu_test
    # F
    _, p_val_t_test = stats.ttest_ind(f0, f1, equal_var=False)
    _, p_val_wwu_test = stats.mannwhitneyu(f0, f1, alternative='two-sided')
    p_val_t_dict["F"] ="%.4f" % p_val_t_test
    p_mwu_val_dict["F"] = "%.4f" % p_val_wwu_test
    # L
    _, p_val_t_test = stats.ttest_ind(l0, l1, equal_var=False)
    _, p_val_wwu_test = stats.mannwhitneyu(l0, l1, alternative='two-sided')
    p_val_t_dict["L"] = "%.4f" % p_val_t_test
    p_mwu_val_dict["L"] = "%.4f" % p_val_wwu_test
    
    print("t-test")
    tb = pt.PrettyTable()
    tb.field_names = p_val_t_dict.keys()
    tb.add_row(p_val_t_dict.values())
    print(tb)
#     print("& ".join(list(p_val_t_dict.values())))
    
    print("WMW-test ")
    tb = pt.PrettyTable()
    tb.field_names = p_mwu_val_dict.keys()
    tb.add_row(p_mwu_val_dict.values())
    print(tb)    
#     print("& " + "& ".join(list(p_mwu_val_dict.values())))
    return avg


In [8]:
def get_diff_combination_scores_and_significant_test(approaches ,data_dir, refs_filename,get_bleu_score):
    all_approaches_result = {}
    for approach in approaches:
        combination_result_dict = {}
        latex_diff_combination_result = []
        for combination in all_data_processing_combinations:
#             refs_filename = os.path.join(data_dir, approach, combination,"test.gold")
            preds_filename = os.path.join(data_dir, approach,combination, "test.pred")
            preds = read_to_list(preds_filename)
            refs = read_to_list(refs_filename)
            refs = [[t] for t in refs[:len( preds)]]
            codenn_bleu = get_bleu_score(refs, preds)
            combination_result_dict[combination] = codenn_bleu
            latex_diff_combination_result.append(round(codenn_bleu,2))
        all_bleu = {key:round(value, 4) for key, value in combination_result_dict.items()}
        print(30*"*")
        print(approach)
        get_t_test_and_MWW_test(list(all_bleu.values()))

In [9]:
get_diff_combination_scores_and_significant_test(approaches ,data_dir,refs_filename,get_bleu_dc)

******************************
codenn
+------+------+------+------+------+------+------+------+
|  R0  |  R1  |  S0  |  S1  |  F0  |  F1  |  L0  |  L1  |
+------+------+------+------+------+------+------+------+
| 7.19 | 7.18 | 7.18 | 7.19 | 7.18 | 7.19 | 7.19 | 7.18 |
+------+------+------+------+------+------+------+------+
t-test
+--------+--------+--------+--------+
|   R    |   S    |   F    |   L    |
+--------+--------+--------+--------+
| 0.9272 | 0.8950 | 0.9098 | 0.8370 |
+--------+--------+--------+--------+
WMW-test 
+--------+--------+--------+--------+
|   R    |   S    |   F    |   L    |
+--------+--------+--------+--------+
| 0.9581 | 0.7929 | 0.9581 | 0.9581 |
+--------+--------+--------+--------+
******************************
astattgru
+------+------+------+------+------+------+------+------+
|  R0  |  R1  |  S0  |  S1  |  F0  |  F1  |  L0  |  L1  |
+------+------+------+------+------+------+------+------+
| 5.91 | 5.97 | 5.63 | 6.26 | 5.85 | 6.03 | 5.81 | 6.07 |
+-

## The Average of five model

In [10]:
def get_four_operations(data):
    r0 = data[8:]
    r1 = data[:8]
    s0 = data[:4] + data[8:12]
    s1 = data[4:8] + data[12:]
    f0 = [data[i] for i in [2, 3, 6, 7, 10, 11, 14, 15]]
    f1 = [data[i] for i in [0, 1, 4, 5, 8, 9, 12, 13]]
    l0 = [data[2 * i] for i in range(8)]
    l1 = [data[2 * i + 1] for i in range(8)]
    return r0, r1, s0, s1, f0, f1, l0, l1


def get_four_opetarot_avg_score(approach_data):
    r0, r1, s0, s1, f0, f1, l0, l1 = [], [], [], [], [], [], [], []
    x1_new, r1_new, s0_new, s1_new, f0_new, f1_new, l0_new, l1_new = get_four_operations(approach_data)
    r0.extend(x1_new) 
    r1.extend(r1_new)
    s0.extend(s0_new)
    s1.extend(s1_new)
    f0.extend(f0_new)
    f1.extend(f1_new)
    l0.extend(l0_new)
    l1.extend(l1_new)
    p_val_t_dict = {} 
    p_mwu_val_dict = {} 
    avg = {"R0":np.mean(r0), "R1":np.mean(r1),"S0":np.mean(s0),"S1":np.mean(s1),
           "F0":np.mean(f0),"F1":np.mean(f1),"L0":np.mean(l0),"L1":np.mean(l1),}
    avg = {key:round(item,2)  for key,item in avg.items()}
    return avg

def get_diff_avg_combination_scores(approaches ,data_dir,refs_filename,get_bleu_score):
    all_approaches_result = {}
    for approach in approaches:
        combination_result_dict = {}
#         latex_diff_combination_result = []
        for combination in all_data_processing_combinations:
#             refs_filename = os.path.join(data_dir, approach, combination,"test.gold")
            preds_filename = os.path.join(data_dir, approach,combination, "test.pred")
            preds = read_to_list(preds_filename)
            refs = read_to_list(refs_filename)
            refs = [[t] for t in refs[:len( preds)]]
            codenn_bleu = get_bleu_score(refs, preds)
            combination_result_dict[combination] = codenn_bleu
#             latex_diff_combination_result.append(round(codenn_bleu,2))
#         all_bleu = {key:round(value, 2) for key, value in combination_result_dict.items()}
#         print(30*"*")
#         print(approach)
        approach_result = get_four_opetarot_avg_score(list(combination_result_dict.values()))
        all_approaches_result[approach] =approach_result
    avg_dict = {}
    for key in all_approaches_result[approach]:
        avg = np.mean([value[key] for value in all_approaches_result.values()])
        avg_dict[key] = round(avg,2)
    print("The average the all model")
    tb = pt.PrettyTable()
    tb.field_names = avg_dict.keys()
    tb.add_row(avg_dict.values())
    print(tb) 
#     print("& " + "& ".join(list(avg_dict.values())))

In [11]:
get_diff_avg_combination_scores(approaches ,data_dir,refs_filename,get_bleu_dc)

The average the all model
+-------+------+-------+-------+-------+-------+-------+-------+
|   R0  |  R1  |   S0  |   S1  |   F0  |   F1  |   L0  |   L1  |
+-------+------+-------+-------+-------+-------+-------+-------+
| 11.79 | 11.7 | 11.34 | 12.15 | 11.72 | 11.77 | 11.56 | 11.92 |
+-------+------+-------+-------+-------+-------+-------+-------+


# Ensemble

In [16]:
def get_ensemble_scores(approaches,data_dir,refs_filename,get_bleu_score):
    result_dict = {}
    for approach in approaches:
        preds_filename = os.path.join(data_dir, approach,"ensemble", "test.pred")
        preds = read_to_list(preds_filename)
        refs = read_to_list(refs_filename)
        refs = [[t] for t in refs[:len( preds)]]
        codenn_bleu = get_bleu_score(refs, preds)
        result_dict[ approach] = codenn_bleu
#             latex_diff_combination_result[combination] = "&%.2f "%codenn_bleu
    all_bleu = {key:round(value, 2) for key, value in result_dict.items()}
    tb = pt.PrettyTable()
    tb.field_names = all_bleu.keys()
    tb.add_row(all_bleu.values())
    print(tb)

In [17]:
get_ensemble_scores(approaches ,data_dir,refs_filename,get_bleu_dc)

+--------+-----------+--------+------+
| codenn | astattgru | rencos | ncs  |
+--------+-----------+--------+------+
| 10.64  |   11.28   | 24.21  | 19.9 |
+--------+-----------+--------+------+
